In [1]:
import requests
from bs4 import BeautifulSoup
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import tkinter as tk
from tkinter import ttk

# scrape in the results
def scrape_electoral_results(url):
    response = requests.get(url)
    if response.status_code != 200:
        return "Failed to retrieve data"
    soup = BeautifulSoup(response.content, 'html.parser')

    # Initialize an empty list to store row data
    data_rows = []

    # Finds the required table and rows within the HTML content
    table = soup.find_all('table')[1]
    rows = table.find_all('tr')[2:53]

    # Loops through the rows to extract data
    for row in rows:
        cells = row.find_all('td')
        state = cells[0].get_text(strip=True).replace('*', '')
        total = int(cells[1].get_text(strip=True))
        trump = cells[2].get_text(strip=True)
        clint = cells[4].get_text(strip=True)
        trump = int(trump) if trump.isdigit() else 0
        clint = int(clint) if clint.isdigit() else 0
        other = total - trump - clint
        color = "red" if trump > clint else "blue"

        # Append the row data to the list
        data_rows.append([state, total, trump, clint, other, color])

    # Create a DataFrame from the list of row data
    election_results_df = pd.DataFrame(data_rows, columns=['State', 'EV Total', 'Trump Total', 'Clinton Total', 'Other Votes', 'Color'])
    election_results_df.set_index('State', inplace=True)

    return election_results_df

# Function to remove specified states from the dataframe
def remove_states(election_results_df, state_names):
    new_results_df = election_results_df.copy()
    for state_name in state_names:
        if state_name in new_results_df.index:
            new_results_df.loc[state_name, ['EV Total', 'Trump Total', 'Clinton Total', 'Other Votes']] = 0
            new_results_df.loc[state_name, 'Color'] = 'white'
        else:
            print(f"FFS broh, '{state_name}' does not exist in the DataFrame.")
    return new_results_df

# Function to plot the results on a map
def map_results(election_results_df, shapefile_path='cb_2018_us_state_500k.shp'):
    # Load the shapefile for US states
    gdf = gpd.read_file(shapefile_path)
    # Filter out the states not present in the election results
    gdf = gdf[gdf['NAME'].isin(election_results_df.index)].copy().reset_index(drop=True)
    # Merge the election results with the geodataframe
    gdf = gdf.merge(election_results_df, how='left', left_on='NAME', right_index=True)

    # Define the Albers Equal Area projection
    aea_proj = '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=37.5 +lon_0=-96'
    # Project the geodataframe to the defined projection
    gdf = gdf.to_crs(aea_proj)

    # Adjust Alaska and Hawaii for visualization purposes
    gdf.loc[gdf['NAME'] == 'Alaska', 'geometry'] = gdf.loc[gdf['NAME'] == 'Alaska'].scale(xfact=0.35, yfact=0.35, origin=(0, 0))
    gdf.loc[gdf['NAME'] == 'Alaska', 'geometry'] = gdf.loc[gdf['NAME'] == 'Alaska'].translate(-1600000, -2500000)
    gdf.loc[gdf['NAME'] == 'Hawaii', 'geometry'] = gdf.loc[gdf['NAME'] == 'Hawaii'].translate(5400000, -1600000)

    # Plot the map
    fig, ax = plt.subplots(1, figsize=(12, 12))
    gdf.boundary.plot(ax=ax, linewidth=1, color='Black')
    gdf.plot(ax=ax, color=gdf['Color'])
    ax.axis('off')
    plt.show()

def bar_results(election_results_df):
    # Calculate the electoral votes needed to win
    ev_to_win = (election_results_df['EV Total'].sum()) // 2 + 1

    # Sum Trump's and Hillary's totals
    trump_total = election_results_df['Trump Total'].sum()
    hillary_total = election_results_df['Clinton Total'].sum()

    # winner and vote totals
    if trump_total > hillary_total:
      winner, winner_votes = "Trump", trump_total
    else:
      winner, winner_votes = "Clinton", hillary_total

    # Data for plotting
    candidates = ['Trump', 'Hillary']
    votes = [trump_total, hillary_total]

    fig, ax = plt.subplots()

    # Create bar chart
    ax.bar(candidates, votes, color=['red', 'blue'])

    # Add a line for the electoral votes needed to win
    ax.axhline(y=ev_to_win, color='k', linestyle='--')
    ax.text(1.02, ev_to_win, f'Majority of ({ev_to_win}) required to win',
            va='center', ha="left", bbox=dict(facecolor="white", alpha=0.5),
            transform=ax.get_yaxis_transform())

    # Add labels and title
    ax.set_ylabel('Total Electoral Votes')
    ax.set_title(f"{winner} wins with {winner_votes} electoral votes")

    # Display the numbers on top of the bars
    for i, v in enumerate(votes):
        ax.text(i, v + 0.2, str(v), color='black', ha='center')

    # Show the plot
    plt.show()

# This function displays the results either as a map or a bar chart based on the user's choice.
def display_results(election_results_df, result_type):
    if result_type == 'map':
        map_results(election_results_df)
    elif result_type == 'bar':
        bar_results(election_results_df)
    plt.show()

# This function runs the Tkinter GUI for state selection and result type choice.
def run_gui(election_results_df):
    # Called when the user clicks the submit button. It processes the selected states
    # and the result type, updates the data frame, displays the results, and closes the GUI.
    def submit():
        selected_states = [state for state, var in checkboxes.items() if var.get()]
        result_type = result_var.get()
        updated_df = remove_states(election_results_df, selected_states)
        display_results(updated_df, result_type)
        root.destroy()

    # Initialize the main window of the GUI.
    root = tk.Tk()
    root.title("Election Results Visualization")

    # Create a dictionary to store the checkboxes for each state.
    checkboxes = {}
    state_list = list(election_results_df.index)
    columns = 3  # Number of columns for checkbox layout
    for i, state in enumerate(state_list):
        var = tk.BooleanVar()
        chk = ttk.Checkbutton(root, text=state, variable=var)
        chk.grid(row=i // columns, column=i % columns, sticky='w')  # Position checkbox in grid
        checkboxes[state] = var

    # Create a variable to store the result type and radio buttons for the user to choose.
    result_var = tk.StringVar(value='map')
    ttk.Radiobutton(root, text="Map Results", variable=result_var, value='map').grid(row=len(state_list) // columns + 1, column=0, sticky='w')
    ttk.Radiobutton(root, text="Bar Results", variable=result_var, value='bar').grid(row=len(state_list) // columns + 1, column=1, sticky='w')

    # Add a submit button to the GUI.
    submit_button = ttk.Button(root, text="Submit", command=submit)
    submit_button.grid(row=len(state_list) // columns + 2, column=0, columnspan=columns, pady=20)

    # Start the main event loop of the Tkinter application.
    root.mainloop()

# The main execution block that runs when the script is called directly.
if __name__ == "__main__":
    url = "https://www.archives.gov/electoral-college/2016"
    election_results_df = scrape_electoral_results(url)
    run_gui(election_results_df)

TclError: no display name and no $DISPLAY environment variable